In [16]:
import numpy as np
import pandas as pd

# Multi-armed Bandits

## Stationary problem with $\epsilon$-greedy strategy and incremental implementation

You are faced with $k$ bandits. Each has a probability $p$ of giving you a reward. You must maximize the total reward that you get over some period (say, 1000 steps).

We don't know a-priori the probability distribution of the rewards for the machines, so we need to balance *exploitation* of the knowledge we already have and *exploration* of alternatives to get a better estimate of the rewards we can get from other machines.

We implement the problem using an $\epsilon$-greedy method and an incremental implementation.

In [17]:
ARMS = 10   # N. of arms
epsilon = 0.1   # For epsilon-greedy search
MAX_STEPS = 1000

rewards = np.random.normal(loc=0.0, scale=1.0, size=(ARMS))     # The actual rewards of the machines
estimates = np.ones((ARMS)) * 5        # Our initial estimate for the rewards: 
                                       # it is a greatly optimistic value so that 
                                       # we can incentivize exploration at the beginning
choice_counter = np.zeros((ARMS))      # How many times we chose each action

We select a choice and update our reward estimates like this:

In [18]:
def choose_bandit(estimates, epsilon) -> int:
    # Choose which bandit to execute with an epsilon-greedy method
    if np.random.random() > epsilon:
        # Choose best estimate
        return np.argmax(estimates)
    else:
        # Choose randomly
        return np.random.randint(0, len(estimates))

def get_reward(rewards, choice) -> float:
    return rewards[choice]

def update_estimates(estimates, choice_counter, choice, reward):
    # Incremental implementation: Qn+1 = Qn + 1/n[Rn-Qn]
    Qn = estimates[choice]
    estimates[choice] = Qn + 1/choice_counter[choice] * (reward - Qn)

Initial situation:

In [19]:
results = pd.DataFrame.from_dict({
    'bandits': list(range(ARMS)),
    'rewards': rewards,
    'n. selected': choice_counter,
    'estimates': estimates
})

results

,bandits,rewards,n. selected,estimates
0,0,0.262664,0.0,5.0
1,1,1.054247,0.0,5.0
2,2,0.218410,0.0,5.0
3,3,0.767939,0.0,5.0
4,4,-2.369840,0.0,5.0
5,5,0.383698,0.0,5.0
6,6,0.491416,0.0,5.0
7,7,-0.642989,0.0,5.0
8,8,0.846402,0.0,5.0
9,9,-0.218943,0.0,5.0


Simulation execution:

In [20]:
total_reward = 0

for i in range(MAX_STEPS):
    action = choose_bandit(estimates, epsilon)
    choice_counter[action] += 1
    r = get_reward(rewards, action)
    total_reward += r
    update_estimates(estimates, choice_counter, action, r)

Final situation:

In [22]:
results = pd.DataFrame.from_dict({
    'bandits': list(range(ARMS)),
    'rewards': rewards,
    'n. selected': choice_counter,
    'estimates': estimates
})

display(results)
print("Total reward: " + str(total_reward))

,bandits,rewards,n. selected,estimates
0,0,0.262664,11.0,0.262664
1,1,1.054247,894.0,1.054247
2,2,0.218410,13.0,0.218410
3,3,0.767939,9.0,0.767939
4,4,-2.369840,15.0,-2.369840
5,5,0.383698,9.0,0.383698
6,6,0.491416,18.0,0.491416
7,7,-0.642989,9.0,-0.642989
8,8,0.846402,13.0,0.846402
9,9,-0.218943,9.0,-0.218943


Total reward: 935.1336677951227


## Non-stationary problem

In this case, the rewards given by the machines can change. We give more weight to recent updates than to older updates by using a fixed step size $\alpha$ (because in this way the previous reward is worth $\alpha^2$ in the incremental update).

In [26]:
ARMS = 10   # N. of arms
epsilon = 0.1   # For epsilon-greedy search
alpha = 0.8
MAX_STEPS = 1000
CHANGE_REWARDS_STEP_MEAN = 100
CHANGE_REWARDS_STEP_SPREAD = 10

def initialize_rewards(loc, scale, size):
    return np.random.normal(loc, scale, size=size) 

def next_reward_change() -> int:
    return int(np.random.normal(CHANGE_REWARDS_STEP_MEAN, CHANGE_REWARDS_STEP_SPREAD))

def update_estimates(estimates, alpha, choice, reward):
    # Incremental implementation: Qn+1 = Qn + alpha[Rn-Qn]
    Qn = estimates[choice]
    estimates[choice] = Qn + alpha * (reward - Qn)

def display_dataframe():
    results = pd.DataFrame.from_dict({
        'bandits': list(range(ARMS)),
        'rewards': rewards,
        'n. selected': choice_counter,
        'estimates': estimates
    })

    display(results)


# Initialization
rewards = initialize_rewards(0.0, 1.0, (ARMS))    # The initial actual rewards of the machines
estimates = np.ones((ARMS)) * 5        # Our initial estimate for the rewards: 
                                       # it is a greatly optimistic value so that 
                                       # we can incentivize exploration at the beginning
choice_counter = np.zeros((ARMS))      # How many times we chose each action
total_reward = 0
next_change = next_reward_change()

print("Initial situation:")
display_dataframe()
print("Simulating game...")
for i in range(MAX_STEPS):
    if i >= next_change:
        print("Epoch {}: changed rewards...".format(i))
        # Change rewards
        rewards = initialize_rewards(0.0, 1.0, (ARMS))
        # Choose next change epoch
        next_change = i + next_reward_change()
    action = choose_bandit(estimates, epsilon)
    choice_counter[action] += 1
    r = get_reward(rewards, action)
    total_reward += r
    update_estimates(estimates, alpha, action, r)
print("Final situation:")
display_dataframe()
print(total_reward)

Initial situation:


,bandits,rewards,n. selected,estimates
0,0,-0.119207,0.0,5.0
1,1,-0.504467,0.0,5.0
2,2,-0.761121,0.0,5.0
3,3,0.711644,0.0,5.0
4,4,0.756229,0.0,5.0
5,5,1.531422,0.0,5.0
6,6,-0.484054,0.0,5.0
7,7,-0.035459,0.0,5.0
8,8,1.029007,0.0,5.0
9,9,-0.168179,0.0,5.0


Simulating game...
Epoch 103: changed rewards...
Epoch 195: changed rewards...
Epoch 292: changed rewards...
Epoch 411: changed rewards...
Epoch 519: changed rewards...
Epoch 618: changed rewards...
Epoch 735: changed rewards...
Epoch 837: changed rewards...
Epoch 938: changed rewards...
Final situation:


,bandits,rewards,n. selected,estimates
0,0,0.095715,82.0,-1.601999
1,1,-1.647242,9.0,-0.631993
2,2,0.580656,8.0,0.200765
3,3,0.575796,40.0,-1.565056
4,4,-0.306784,250.0,-0.772785
5,5,1.044041,130.0,-1.382450
6,6,0.783652,61.0,0.783652
7,7,-0.438579,338.0,-0.434300
8,8,1.288995,18.0,1.288976
9,9,-0.956265,64.0,-0.908827


806.9343449437534
